In [35]:
import geopandas as gpd
import pandas as pd
import glob
from shapely.geometry import MultiPoint, MultiLineString, LineString, Point
from shapely.ops import split, unary_union, snap, linemerge
from IPython.display import clear_output, display
import numpy as np
import pygeos as pg

In [3]:
distritos = gpd.read_file(f'downloads/SIRGAS_GPKG_distrito.gpkg')

In [51]:
# distritos[distritos.ds_codigo == '22']
## TODO
# Distritos 22 e 52 (Cid. Ademar e Marsilac) não processaram, verificar!

In [47]:
for i, d in distritos.iterrows():

    arquivo_resultante = f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg'

    if not glob.glob(arquivo_resultante):
        folder = f'{d.ds_codigo} - {d.ds_nome}'
        clear_output(wait=True)
        print(folder)

        faixas_poly = glob.glob(f'resultados/vetor/{folder}/faixas-poly-*.gpkg')
        if not len(faixas_poly): break
        gdf_faixas_poly = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in faixas_poly]))
        gdf_faixas_poly = gdf_faixas_poly.set_crs(epsg=31983)

        linhas_de_travessia = glob.glob(f'resultados/vetor/{folder}/linhas-de-travessia-*.gpkg')
        gdf_linhas_de_travessia = gpd.GeoDataFrame(pd.concat([gpd.read_file(f) for f in linhas_de_travessia]))
        gdf_linhas_de_travessia = gdf_linhas_de_travessia.set_crs(epsg=31983)

        gdf_rotas_mediais = gpd.read_file(f'../rotas-em-calcadas/resultados/eixos_mediais/{d.ds_codigo}-eixos-mediais-{d.ds_nome}.gpkg')
        gdf_calcadas = gpd.read_file(f'../rotas-em-calcadas/resultados/calcadas/{d.ds_codigo}-calcadas-{d.ds_nome}.gpkg')

        # define a function that rounds the coordinates of every geometry in the array
        round = np.vectorize(lambda geom: pg.apply(geom, lambda g: g.round(3)))
        gdf_linhas_de_travessia.geometry = round(gdf_linhas_de_travessia.geometry.values.data)
        gdf_rotas_mediais.geometry = round(gdf_rotas_mediais.geometry.values.data)

        rotas_caminhaveis = gdf_linhas_de_travessia.unary_union.union(gdf_rotas_mediais.unary_union)
        geometrias = [i for i in linemerge(rotas_caminhaveis)]

        gdf_rotas_caminhaveis = gpd.GeoDataFrame(geometry=geometrias).set_crs(epsg=31983)
        gdf_rotas_caminhaveis.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
        driver='GPKG', layer='rede_caminhavel')
        gdf_faixas_poly.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
        driver='GPKG', layer='faixas_poly')
        gdf_linhas_de_travessia.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
        driver='GPKG', layer='linhas_de_travessia')
        gdf_rotas_mediais.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
        driver='GPKG', layer='rotas_mediais')
        gdf_calcadas.to_file(f'resultados/preliminar/rede-rotas-preliminar-{d.ds_codigo}-{d.ds_nome}.gpkg', 
        driver='GPKG', layer='calcadas')



52 - MARSILAC
